In [1]:
import requests
import time
import pandas as pd
import numpy as np
import re
import json
import pdb
from bs4 import BeautifulSoup
import personal_keys
import ravelry_request_functions

In [2]:
# multiple pattern request-
# input: list of patterns, output: json file with patterns
def multiple_pattern_request(pattern_list):
    patterns_url = 'https://api.ravelry.com/patterns.json?ids={}'.format('+'.join(pattern_list))
    patterns = requests.get(patterns_url, 
                            auth = (personal_keys.username(),personal_keys.password()))
    if patterns.status_code is 200:
        return patterns.json()
    else:
        return 404

# input: pattern json from the multiple pattern request function
def create_attr_list(pattern_list):
    attr_list = []
    patterns = multiple_pattern_request(pattern_list)
    if patterns != 404:
        for key in patterns['patterns'].keys():
            attr_list.append({attr['permalink']:1 for attr in 
            patterns['patterns'][key]['pattern_attributes']})
    return attr_list

# input: list of pattern id's, 
# output: dictionary where keys are pattern id's and values are a dict of attributes
def pattern_attr(pattern_list):
    pattern_list = [str(item) for item in pattern_list]
    if len(pattern_list) < 50:
        attr_list = create_attr_list(pattern_list)
    else:
#         create nested list that contains lists of either 50 patterns or the remainder of length of list/50
        l_of_l_patterns = [pattern_list[i:i + 50] for i in range(0, len(pattern_list), 50)]
        batch_num = 0
        attr_list = []
        while batch_num < len(l_of_l_patterns):
            batch_attr_list = create_attr_list(l_of_l_patterns[batch_num])
            attr_list.extend(batch_attr_list)
            batch_num += 1
    return dict(zip(pattern_list, attr_list))

In [3]:
test_list = ravelry_request_functions.get_favs_list("swiftmiss")

In [4]:
len(test_list)

462

In [7]:
test_list

[1002216,
 1002226,
 982422,
 987228,
 984587,
 984587,
 984587,
 707297,
 927223,
 203482,
 384733,
 987452,
 3418,
 962172,
 794359,
 142296,
 922283,
 870909,
 695625,
 695622,
 981622,
 984587,
 906148,
 133526,
 709752,
 976824,
 2511,
 972099,
 182135,
 853291,
 787476,
 958384,
 946729,
 942286,
 953241,
 819716,
 767529,
 599849,
 464784,
 958636,
 956565,
 824443,
 935606,
 954279,
 882501,
 477615,
 845213,
 776573,
 672418,
 916579,
 748023,
 945853,
 745177,
 945667,
 943797,
 932897,
 20251224,
 932709,
 933928,
 940692,
 942137,
 937550,
 927223,
 819716,
 882927,
 939348,
 770956,
 932897,
 869199,
 927223,
 697567,
 920062,
 925402,
 922543,
 921761,
 610717,
 575037,
 884470,
 905853,
 551182,
 825925,
 768791,
 735180,
 846954,
 675502,
 885091,
 877380,
 917403,
 886287,
 1843,
 911227,
 565793,
 909127,
 815785,
 910203,
 829380,
 895744,
 547629,
 692873,
 705977,
 908249,
 887934,
 841735,
 352534,
 873567,
 906203,
 906203,
 823476,
 893868,
 552949,
 902002,
 74

In [5]:
test_dict = pattern_attr(test_list)

In [6]:
len(test_dict.keys())

176